# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
# Import API key
from config import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../Resources/weather.csv')
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kavieng,-2.5744,150.7967,81.16,76,71,7.49,PG,1615823932
1,aksu,41.1231,80.2644,58.01,28,100,14.79,CN,1615823932
2,bluff,-46.6000,168.3333,51.01,92,89,7.00,NZ,1615823932
3,ponta delgada,38.7333,-28.2667,62.60,88,40,9.22,PT,1615823933
4,jamestown,42.0970,-79.2353,28.99,35,1,10.31,US,1615823933


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)

In [4]:
# Lat and lng as locations
locations = weather_df[["Lat", "Lng"]]


humidity = weather_df["Humidity"].astype(float)


fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Humidity as the weight
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=np.max(humidity),
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# # Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions.

ideal_weather_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                  (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()
ideal_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,puerto ayora,36.5939,-6.2330,73.40,40,0,3.44,ES,1615823933
28,port lincoln,-7.0349,29.7638,78.40,68,0,2.10,CD,1615823941
46,port elizabeth,-7.0349,29.7638,78.40,68,0,2.10,CD,1615823941
59,kalmunai,7.4167,81.8167,79.12,79,0,6.17,LK,1615823950
101,port alfred,-7.0349,29.7638,78.40,68,0,2.10,CD,1615823963


In [6]:
# Be sure to limit the number of rows returned by your API requests to a reasonable number.
ideal_weather_df.count()

City          20
Lat           20
Lng           20
Max Temp      20
Humidity      20
Cloudiness    20
Wind Speed    20
Country       20
Date          20
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame(ideal_weather_df, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
6,puerto ayora,ES,36.5939,-6.2330,
28,port lincoln,CD,-7.0349,29.7638,
46,port elizabeth,CD,-7.0349,29.7638,
59,kalmunai,LK,7.4167,81.8167,
101,port alfred,CD,-7.0349,29.7638,
127,port keats,CD,-7.0349,29.7638,
209,port augusta,CD,-7.0349,29.7638,
256,port elgin,CD,-7.0349,29.7638,
257,port blair,CD,-7.0349,29.7638,
278,port hedland,CD,-7.0349,29.7638,


In [8]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : gkey}

In [9]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for:{city}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        print("-------------------------")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
print("--------------------Search Completed")

Retrieving Results for:puerto ayora.
Closest hotel in puerto ayora is Duques de Medinaceli.
-------------------------
Retrieving Results for:port lincoln.
Could not find result
-------------------------
Retrieving Results for:port elizabeth.
Could not find result
-------------------------
Retrieving Results for:kalmunai.
Closest hotel in kalmunai is LUXEZ garden house.
-------------------------
Retrieving Results for:port alfred.
Could not find result
-------------------------
Retrieving Results for:port keats.
Could not find result
-------------------------
Retrieving Results for:port augusta.
Could not find result
-------------------------
Retrieving Results for:port elgin.
Could not find result
-------------------------
Retrieving Results for:port blair.
Could not find result
-------------------------
Retrieving Results for:port hedland.
Could not find result
-------------------------
Retrieving Results for:port hawkesbury.
Could not find result
-------------------------
Retrieving 

In [10]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
6,puerto ayora,ES,36.5939,-6.2330,Duques de Medinaceli
28,port lincoln,CD,-7.0349,29.7638,
46,port elizabeth,CD,-7.0349,29.7638,
59,kalmunai,LK,7.4167,81.8167,LUXEZ garden house
101,port alfred,CD,-7.0349,29.7638,
127,port keats,CD,-7.0349,29.7638,
209,port augusta,CD,-7.0349,29.7638,
256,port elgin,CD,-7.0349,29.7638,
257,port blair,CD,-7.0349,29.7638,
278,port hedland,CD,-7.0349,29.7638,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))